In [1]:
import csv
import requests
import json
import random

In [178]:
def search(query):
    relevant_docs = []
    with open(r'C:\Users\NEW\Downloads\cran\cranqrel', 'r') as rel:
        csv_read = csv.reader(rel)
        for row in csv_read:
            q_no, doc_id, rel_no = row[0].split()
            if int(q_no) == 2:
                if int(rel_no)>=2:
                    relevant_docs.append(doc_id)
    print(relevant_docs) 
    
    precision_array = []
    map_array = []
    for a in range(10):
        for b in range(10):
            title = query['query']['multi_match']['fields'][0].split('^')
            title[1] = str(a)
            title = '^'.join(title)
            query['query']['multi_match']['fields'][0] = title
            data = query['query']['multi_match']['fields'][1].split('^')
            data[1] = str(b)
            data = '^'.join(data)
            query['query']['multi_match']['fields'][1] = data
                     
            url = 'http://localhost:9200/cran_experiment/_search'
            httpResp = requests.get(url, data=json.dumps(query), headers={"content-type":"application/json"})
            searchHits = json.loads(httpResp.text)["hits"]
            rel_docs = 0
            c = 0
            precision = []
            for idx, hit in enumerate(searchHits['hits']):
                if hit['_source']['id'] in relevant_docs:
                    print('\n')
                    rel_docs += 1
                c += 1
                precision.append(rel_docs/c)
        precision_array.append([sum(precision), a, b])
            
    map_array = [x[0]/10 for x in precision_array]
    print(len(map_array))
    print(f"The MAP Score of the corpus on the given query is: {max(map_array)}")
    
    print(f"The optimized weights are: ")
    for i in range(len(map_array)):
        if map_array[i] == max(map_array):
            a = precision_array[i][1]
            b = precision_array[i][2]
            title = query['query']['multi_match']['fields'][0].split('^')
            title[1] = str(a)
            title = '^'.join(title)
            query['query']['multi_match']['fields'][0] = title
            data = query['query']['multi_match']['fields'][1].split('^')
            data[1] = str(b)
            data = '^'.join(data)
            query['query']['multi_match']['fields'][1] = data
            url = 'http://localhost:9200/cran_experiment/_search'
            httpResp = requests.get(url, data=json.dumps(query), headers={"content-type":"application/json"})
            searchHits = json.loads(httpResp.text)["hits"]
            print ("Num\tRelevance Score\t\t ID\t Title")
            for idx, hit in enumerate(searchHits['hits']):
                print("%s\t%s\t\t%s\t%s" %
                      (idx + 1, hit['_score'], hit['_source']['id'], hit['_source']['title']))
            break

In [179]:
query = {      
    "query": {
        "multi_match": {
            "query": "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft",
            "fields": [f"title^1", f"data^1"]
         }
     }
}

In [180]:
search(query)

['15', '184', '858', '51', '102', '202', '14', '52', '380', '859', '948', '285', '390', '391', '442', '497', '643', '856', '857', '877', '864', '658']
























































































































































































































































































































































































































































































































10
The MAP Score of the corpus on the given query is: 0.39107142857142857
The optimized weights are: 
Num	Relevance Score		 ID	 Title
1	207.9244		184	scale models for thermo-aeroelastic research .
2	188.16914		486	similarity laws for aerothermoelastic testing .
3	176.17871		13	similarity laws for stressing heated wings .
4	164.00404		1268	stab

In [174]:

query = {      
    "query": {
        "match": {
            "title": "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft"
         }
     }
}

In [177]:
search(query)

['15', '184', '858', '51', '102', '202', '14', '52', '380', '859', '948', '285', '390', '391', '442', '497', '643', '856', '857', '877', '864', '658']
























































































































































































































































































































































































































































































































10
The MAP Score of the corpus on the given query is: 0.39107142857142857
The optimized weights are: 
Num	Relevance Score		 ID	 Title
1	207.9244		184	scale models for thermo-aeroelastic research .
2	188.16914		486	similarity laws for aerothermoelastic testing .
3	176.17871		13	similarity laws for stressing heated wings .
4	164.00404		1268	stab

In [72]:
query = {      
    "query": {
        "multi_match": {
            "query": "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft",
            "fields": ["title^1", "data^10"]
         }
     }
}

In [73]:
search(query)

['51', '102', '202', '14', '52', '380', '285', '390', '391', '442', '497', '643', '856', '857', '877', '864', '658']
{'query': {'multi_match': {'query': 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft', 'fields': ['title^0', 'data^10']}}}
Num	Relevance Score		 ID	 Title
1	0.0		1	experimental investigation of the aerodynamics of a
wing in a slipstream .
2	0.0		2	simple shear flow past a flat plate in an incompressible fluid of small
viscosity .
3	0.0		4	approximate solutions of the incompressible laminar
boundary layer equations for a plate in shear flow .
4	0.0		5	one-dimensional transient heat conduction into a double-layer
slab subjected to a linear heat input for a small time
internal .
5	0.0		6	one-dimensional transient heat flow in a multilayer
slab .
6	0.0		7	the effect of controlled three-dimensional roughness
on boundary layer transition at supersonic speeds .
7	0.0		8	measurements of the effect of two-dimensional and thre

Num	Relevance Score		 ID	 Title
1	92.410835		184	scale models for thermo-aeroelastic research .
2	83.63073		486	similarity laws for aerothermoelastic testing .
3	78.30165		13	similarity laws for stressing heated wings .
4	72.890686		1268	stable combustion of a high-velocity gas in a heated
boundary layer .
5	71.60799		12	some structural and aerelastic considerations of high
speed flight .
6	59.552963		51	theory of aircraft structural models subjected to aerodynamic
heating and external loads .
7	56.229336		878	experimental model techniques and equipment for flutter
investigations .
8	56.102737		14	piston theory - a new aerodynamic tool for the
aeroelastician .
9	49.482685		1361	large deflections of structures subjected to heating
and external loads .
10	48.564457		172	some aerodynamic considerations of nozzle afterbody
combination .
{'query': {'multi_match': {'query': 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft', 'fields': ['t

Num	Relevance Score		 ID	 Title
1	161.71896		184	scale models for thermo-aeroelastic research .
2	146.35378		486	similarity laws for aerothermoelastic testing .
3	137.0279		13	similarity laws for stressing heated wings .
4	127.5587		1268	stable combustion of a high-velocity gas in a heated
boundary layer .
5	125.31397		12	some structural and aerelastic considerations of high
speed flight .
6	104.21769		51	theory of aircraft structural models subjected to aerodynamic
heating and external loads .
7	98.401344		878	experimental model techniques and equipment for flutter
investigations .
8	98.179794		14	piston theory - a new aerodynamic tool for the
aeroelastician .
9	86.5947		1361	large deflections of structures subjected to heating
and external loads .
10	84.9878		172	some aerodynamic considerations of nozzle afterbody
combination .
{'query': {'multi_match': {'query': 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft', 'fields': ['titl

3	156.6033		13	similarity laws for stressing heated wings .
4	145.78137		1268	stable combustion of a high-velocity gas in a heated
boundary layer .
5	143.21597		12	some structural and aerelastic considerations of high
speed flight .
6	119.10593		51	theory of aircraft structural models subjected to aerodynamic
heating and external loads .
7	112.45867		878	experimental model techniques and equipment for flutter
investigations .
8	112.205475		14	piston theory - a new aerodynamic tool for the
aeroelastician .
9	98.96537		1361	large deflections of structures subjected to heating
and external loads .
10	97.128914		172	some aerodynamic considerations of nozzle afterbody
combination .
{'query': {'multi_match': {'query': 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft', 'fields': ['title^6', 'data^8']}}}
Num	Relevance Score		 ID	 Title
1	207.9244		184	scale models for thermo-aeroelastic research .
2	188.16914		486	similarity laws for aerot

10	76.13058		1111	some research on high speed flutter .
{'query': {'multi_match': {'query': 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft', 'fields': ['title^9', 'data^1']}}}
Num	Relevance Score		 ID	 Title
1	188.03912		13	similarity laws for stressing heated wings .
2	134.84409		486	similarity laws for aerothermoelastic testing .
3	127.625885		746	aeroelastic problems in connection with high speed
flight .
4	127.15917		792	some low speed problems of high speed aircraft .
5	125.76528		875	models for aeroelastic investigation .
6	114.90327		184	scale models for thermo-aeroelastic research .
7	80.73757		51	theory of aircraft structural models subjected to aerodynamic
heating and external loads .
8	79.56656		1250	high-speed viscous corner flow .
9	78.460205		1268	stable combustion of a high-velocity gas in a heated
boundary layer .
10	76.13058		1111	some research on high speed flutter .
{'query': {'multi_match': {'query': 'what sim